In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [92]:
dataset = pd.read_csv("C:\\SparkCourse\\twitter\\twitter_new.csv", encoding='latin-1', header=None)

In [99]:
dataset.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [102]:
dataset.shape

(4137, 6)

In [103]:
dataset[0].value_counts()

0    2103
4    2034
Name: 0, dtype: int64

In [104]:
dataset[2103:].head()

,0,1,2,3,4,5
2103,4,1467822272,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,ersle,I LOVE @Health4UandPets u guys r the best!!
2104,4,1467822273,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,becca210,im meeting up with one of my besties tonight! ...
2105,4,1467822283,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,Wingman29,"@DaRealSunisaKim Thanks for the Twitter add, S..."
2106,4,1467822287,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,katarinka,Being sick can be really cheap when it hurts t...
2107,4,1467822293,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,_EmilyYoung,@LovesBrooklyn2 he has that effect on everyone


In [97]:
np.unique(dataset[0])

array([0, 4], dtype=int64)

In [85]:
dataset[0].value_counts()

4    800000
0    800000
Name: 0, dtype: int64

In [106]:
#df = pd.DataFrame([dataset[5],dataset[0]], columns=['text','sentiment'])
df = dataset.loc[:, [0,5]]

In [107]:
df.shape

(4137, 2)

In [89]:
dataset.where(dataset[0]==4).head()

,0,1,2,3,4,5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df = df.append(dataset.where(dataset[0]==4).iloc[:2000, [0,5]])

In [108]:
df[0].value_counts()

0    2103
4    2034
Name: 0, dtype: int64

In [109]:
df = df.rename(columns={0:"sentiment", 5:"text"})

In [110]:
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [111]:
df_text = df['text']
df_sentiment = df['sentiment']

In [112]:
df.insert(0, "text_new", df_text)

In [113]:
df.insert(1, "sentiment_new", df_sentiment)

In [114]:
df.drop(['sentiment','text'], axis=1, inplace=True)

In [115]:
df.head()

,text_new,sentiment_new
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [116]:
df = df.rename(columns={"text_new":"text", "sentiment_new":"sentiment"})

In [117]:
df.head()

,text,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [119]:
df.where(df['sentiment']==4).count()

text         2034
sentiment    2034
dtype: int64

In [120]:
df.shape

(4137, 2)

In [121]:
df['text'][:1]

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
Name: text, dtype: object

In [23]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to C:\Users\Alok
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [122]:
corpus = []
for i in range(0, len(df)):
    #tweet = re.sub('[@\w+\s]', ' ', df['text'][i])
    #tweet = re.sub('[^a-zA-Z]', ' ', df['text'][i])
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",df['text'][i]).split())
    tweet = tweet.lower()
    tweet = tweet.split()
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet if word not in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

In [123]:
len(corpus)

4137

In [124]:
corpus[1]

'upset updat facebook text might cri result school today also blah'

In [125]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus)
y = df.iloc[:, -1]

In [126]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [130]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train.toarray(), y_train)
y_pred = nb.predict(X_test.toarray())

In [132]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[221 324]
 [106 384]]


0.5845410628019324

In [133]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf')
svc.fit(X_train.toarray(), y_train)
y_pred = svc.predict(X_test.toarray())

In [134]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[356 189]
 [145 345]]


0.6772946859903382

In [135]:
from sklearn.ensemble import RandomForestClassifier
rbf = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
rbf.fit(X_train.toarray(), y_train)
y_pred = rbf.predict(X_test.toarray())

In [136]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[364 181]
 [191 299]]


0.6405797101449275

In [137]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0)
lr.fit(X_train.toarray(), y_train)
y_pred = lr.predict(X_test.toarray())

In [138]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[359 186]
 [156 334]]


0.6695652173913044